<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/Dementia_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
